### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
sd = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
numstu=sd['Student ID'].count()
sd_grouped= sd.groupby(['school_name'])
school_average = sd_grouped.mean()
numschl = sd['school_name'].nunique()
rdscr = school_average['reading_score'].mean()
psrd = sd['Student ID'].loc[sd['reading_score']>=70].count()
mthscr = school_average['math_score'].mean()
psmth = sd['Student ID'].loc[sd['math_score']>=70].count()
pctrd = round(psrd/numstu,3) * 100
pctmth = round(psmth/numstu,3)*100
pctpas = round((pctrd + pctmth)/2)



## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
ds=pd.DataFrame( {'Total Schools':numschl, 'Total Students':numstu,'Total Budget':school_average['budget'].sum(), 
       'Math:Average Score':mthscr, 'Math:Percent Passed':pctmth,
      'Reading: Average Score':[rdscr], 'Reading:Percent Passed':pctrd, 
     'Percent Passed Avg': pctpas})
ds['Total Budget']=ds['Total Budget'].map("${:,.2f}".format)
ds['Total Students']=ds['Total Students'].map("{:,}".format)
ds.head()



,Total Schools,Total Students,Total Budget,Math:Average Score,Math:Percent Passed,Reading: Average Score,Reading:Percent Passed,Percent Passed Avg
0,15,"39,170","$24,649,428.00",80.43253,75.0,82.529188,85.8,80.0


In [3]:
del school_average['Student ID']
del school_average['School ID']
del school_average['size']
del school_average['budget']
del school_data['School ID']
ss = pd.merge(school_data, school_average, how="left", on=["school_name", "school_name"])
psmth = sd.loc[sd['math_score']>=70]
psmth_byschool = psmth.groupby(['school_name'])['math_score'].count()
psrd = sd.loc[sd['reading_score']>=70]
psrd = psrd.groupby(['school_name'])['reading_score'].count()
ss['Per Student Budget']=ss['budget']/ss['size']
ss = pd.merge(ss, psrd, how="left", on=["school_name", "school_name"])
ss = pd.merge(ss, psmth_byschool, how="left", on=["school_name", "school_name"])
ss['% Passing Math']= ss['math_score_y']/ss['size']*100
ss['% Passing Reading'] = ss['reading_score_y']/ss['size']*100
ss['% Overall Passing Rate'] = (ss['% Passing Math']+ss['% Passing Reading'])/2
ss = ss.rename(columns={'type':'School Type','size':'Total Students', 'budget':'Total School Budget',
    'reading_score_x':'Average Reading Score',  'math_score_x':'Average Math Score'})
del ss['reading_score_y']
del ss['math_score_y']
ss = ss [['school_name','School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 
          'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
         '% Overall Passing Rate']]
ss.head()

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [28]:
top_performing= ss.sort_values('% Overall Passing Rate', ascending = False)
top_performing['Total School Budget']=top_performing['Total School Budget'].map("${:,.2f}".format)
top_performing['Per Student Budget']=top_performing['Per Student Budget'].map("${:.2f}".format)
top_performing=top_performing.set_index('school_name')
top_performing=top_performing.rename_axis(None)
top_performing.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


In [27]:
bottom_performing = ss.sort_values('% Overall Passing Rate')
bottom_performing['Total School Budget']=bottom_performing['Total School Budget'].map("${:,.2f}".format)
bottom_performing['Per Student Budget']=bottom_performing['Per Student Budget'].map("${:.2f}".format)
bottom_performing=bottom_performing.set_index('school_name')
bottom_performing=bottom_performing.rename_axis(None)
bottom_performing.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [37]:
# a = sc.loc[sc['math_score'] >= 70]
# b = a.groupby(['school_name','grade']).size()
# tbg = sc.groupby(['school_name','grade']).size()

# # # perpsmth = 100*psmth/tbg
# # # perpsmth.head(30)
# b.head(50)
sd=sd.rename(columns={'school_name':'School Name','grade':' '})
avg_bygrade = sd.groupby(['School Name',' ']).mean()
del avg_bygrade['School ID']
del avg_bygrade['Student ID']
del avg_bygrade['size']
del avg_bygrade['budget']
avg_bygrade.head()

reading_score  math_score
School Name                                        
Bailey High School  10th      80.907183   76.996772
                    11th      80.945643   77.515588
                    12th      80.912451   76.492218
                    9th       81.303155   77.083676
Cabrera High School 10th      84.253219   83.154506

In [38]:
avg_math = avg_bygrade.copy()
del avg_math['reading_score']
avg_math = avg_math.rename(columns={'math_score':''})
a= avg_math.unstack(1)
a = a.rename_axis(None)
a=a.sort_index(level=[0,1])
a

10th       11th       12th        9th
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [40]:
avg_read = avg_bygrade.copy()
del avg_read['math_score']
avg_read=avg_read.rename(columns={'reading_score':''})
b=avg_read.unstack(1)
b = b.rename_axis(None)
b.head()

10th       11th       12th        9th
Bailey High School    80.907183  80.945643  80.912451  81.303155
Cabrera High School   84.253219  83.788382  84.287958  83.676136
Figueroa High School  81.408912  80.640339  81.384863  81.198598
Ford High School      81.262712  80.403642  80.662338  80.632653
Griffin High School   83.706897  84.288089  84.013699  83.369193

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [34]:
# Sample bins. Feel free to create your own bins.
school_spending = ss.copy()
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
school_spending["Spending Ranges (Per Student)"] = pd.cut(school_spending["Per Student Budget"], spending_bins, labels=group_names, include_lowest=True)
del school_spending['Total Students']
del school_spending['Total School Budget']
del school_spending['Per Student Budget']
school_spending.groupby(['Spending Ranges (Per Student)']).mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [42]:
# Sample bins. Feel free to create your own bins.
school_size = ss.copy()
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_size["School Size"] = pd.cut(school_size["Total Students"], size_bins, labels=group_names, include_lowest=True)
del school_size['Total Students']
del school_size['Total School Budget']
del school_size['Per Student Budget']
school_size.groupby(['School Size']).mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [44]:
school_type = ss.groupby(['School Type']).mean()
del school_type['Total Students']
del school_type['Total School Budget']
del school_type['Per Student Budget']
school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
